**Kaynakça:**
> **https://machinelearningmastery.com/how-to-configure-k-fold-cross-validation/**

* https://yigitsener.medium.com/makine-%C3%B6%C4%9Frenmesinde-cross-validation-ve-python-%C3%BCzerinden-k-fold-uygulamas%C4%B1-c535f24cfbbe
* https://www.section.io/engineering-education/how-to-implement-k-fold-cross-validation/
* https://vitalflux.com/k-fold-cross-validation-python-example/

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_digits
digits = load_digits()

In [2]:
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.3)

## Logistic Regression

In [3]:
lr = LogisticRegression(solver = 'liblinear', multi_class = 'ovr')
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.9629629629629629

## SVM

In [4]:
svm = SVC(gamma='auto')
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.39444444444444443

## Random Forest

In [5]:
rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9740740740740741

## KFold cross validation

In [6]:
# Basit bir örnek üzerinden gidelim.
kf = KFold(n_splits=3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [7]:
for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index,test_index)

[3 4 5 6 7 8] [0 1 2]
[0 1 2 6 7 8] [3 4 5]
[0 1 2 3 4 5] [6 7 8]


In [8]:
# 'digits' örneğimiz için KFold kullanalım
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

**StratifiedKFold**

In [9]:
folds = StratifiedKFold(n_splits=3)

scores_logistic = []
scores_svm = []
scores_rf = []

for train_index, test_index in folds.split(digits.data,digits.target):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                       digits.target[train_index], digits.target[test_index]
    scores_logistic.append(get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test))  
    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))

In [10]:
scores_logistic

[0.8948247078464107, 0.9532554257095158, 0.9098497495826378]

In [11]:
scores_svm

[0.3806343906510851, 0.41068447412353926, 0.5125208681135225]

In [12]:
scores_rf

[0.9282136894824707, 0.9382303839732888, 0.9198664440734557]

## cross_val_score function
> cross_val_score, varsayılan olarak **stratifield kfold**'u kullanır

In [13]:
# cross_val_score kullanarak lojistik regresyon modeli performansı
cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'), digits.data, digits.target,cv=3)

array([0.89482471, 0.95325543, 0.90984975])

In [14]:
# cross_val_score kullanarak svm modeli performansı
cross_val_score(SVC(gamma='auto'), digits.data, digits.target,cv=3)

array([0.38063439, 0.41068447, 0.51252087])

In [15]:
# cross_val_score kullanarak random forest modeli performansı
cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target,cv=3)

array([0.93823038, 0.94490818, 0.92654424])

### KFold Cross doğrulama kullanarak parametre ayarı

In [16]:
scores1 = cross_val_score(RandomForestClassifier(n_estimators=5),digits.data, digits.target, cv=10)
np.average(scores1)

0.872042209807573

In [17]:
scores2 = cross_val_score(RandomForestClassifier(n_estimators=20),digits.data, digits.target, cv=10)
np.average(scores2)

0.9332247051520793

In [18]:
scores3 = cross_val_score(RandomForestClassifier(n_estimators=30),digits.data, digits.target, cv=10)
np.average(scores3)

0.9476939788950961

In [19]:
scores4 = cross_val_score(RandomForestClassifier(n_estimators=40),digits.data, digits.target, cv=10)
np.average(scores4)

0.9488019863438858

Burada, rastgele orman sınıflandırıcımızın ince ayarını yapmak için cross_val_score kullandık ve rastgele ormanda yaklaşık 40 ağaca sahip olmanın en iyi sonucu verdiğini düşündük.